In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "/home/jovyan/Work/ej/paquetes/nlppen/")

from nlppen.analisis import Analisis
from nlppen.seleccion import Seleccion
from nlppen.spark_udfs import solo_portanto
from pyspark.sql import SparkSession



spark = (SparkSession
         .builder
         .appName("Prueba Steven")
         .config("spark.num.executors", "2")
         .config("spark.executor.memory", "5g")
         .config("spark.executor.cores", "1")
         .config("spark.driver.memory", "10g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "20g")
         .config("spark.sql.execution.arrow.pyspark.enabled", "true")
         .getOrCreate())

sc = spark.sparkContext
sc.uiWebUrl

sc.addPyFile("nlppen.zip")

21/09/02 18:02:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


21/09/02 18:01:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [9]:
import pandas as pd
pd.read_parquet('./temp.parquet').columns

Index(['archivo', 'txt', 'tema', 'tema_prob', 'expediente', 'fechahora_ext',
       'redactor_ext', 'tipoResolucion_ext', 'termino_ext', 'voto_salvado',
       'magistrados_ext', 'Fechahora_ents', 'Redactor_ents',
       'CitaSentencias_ents', 'Reglamento_ents', 'termino_lst', 'subtema',
       'Cantón_ents', 'Ley_ents', 'EntidadPublica_ents', 'subtema_prob',
       'Constitución_ents', 'Fecha_ents', 'anno'],
      dtype='object')

In [4]:
%%time
# CPU times: user 23.1 ms, sys: 12.5 ms, total: 35.5 ms
# Wall time: 1min 13s

terminos = {
    'Servicio de Agua': ['Servicios? de Agua'],
    'Derecho al agua': ['Derecho Fundamental al Agua', 'Derecho humano al Agua', 'Derecho al agua'],
    'Contaminación de aguas': ['Contaminaci[óo]n de aguas?'],
    'parrafo': ["Así por ejemplo, podría pensarse en la falta de respuesta a una denuncia planteada por contaminación por aguas negras, o en la omisión en la resolución de una denuncia por contaminación de aguas subterráneas, problemas que en caso de no ser tratados a tiempo podrían generar consecuencias nefastas para la salud de la población."]
}

terminos_sub = {
    'Consulta Facultativa': ['Consulta Facultativa'],
    'Consulta Preceptiva': ['Consulta legislativa', 'Consulta preceptiva']
}

terminos_legalidad = {
    'No corresponde' : [
        'no le corresponde a esta sala',
        'aspecto de legalidad',
        'es ajeno al [áa]mbito de esta sala'
    ],
    'legalidad': ['legalidad']
}

# a = Analisis(terminos, spark, datasets_path='datasets_derArgua/')
s = Seleccion(terminos, spark, parquet_path='./temp.parquet')
s.sdf.count()
s.sdf.rdd.take(5)

CPU times: user 13 ms, sys: 1.33 ms, total: 14.3 ms
Wall time: 1.54 s


[Row(archivo='77971.html', txt='*060009440007CO* Exp: 06-000944-0007-CO Res. Nº 2006-02643 SALA CONSTITUCIONAL DE LA CORTE SUPREMA DE JUSTICIA. San José, a las diecisiete horas treinta y cuatro minutos del veintiocho de febrero del dos mil seis.\nRecurso de amparo interpuesto por HELENA CASTILLO RAMIREZ, mayor, portadora de la cédula de identidad número 1-858-166, contra la FISCALIA DEL COLEGIO DE ABOGADOS.\nResultando:\n1.- Por escrito recibido en la Secretaría de la Sala a las 13:05 horas del 27 de enero del 2006, la recurrente en su contra se tramita un procedimiento administrativo disciplinario (expediente 138-04), que se inició con un auto de apertura que no contenía una descripción clara, concisa y circunstanciada de los hechos intimados, y además no se indicó a cuál de las faltas contenidas en la normativa interna del Colegio supuestamente correspondían los hechos imputados, con lo que se infringió el principio de intimación e imputación. Así se acusó en el procedimiento, pero n

In [11]:
s.filtrar_sentencias()
print('OK')

OK


In [15]:
legalidad = s.sub_busqueda(terminos_legalidad, preprocess=solo_portanto)
legalidad.columns

['archivo',
 'txt',
 'tema',
 'tema_prob',
 'expediente',
 'fechahora_ext',
 'redactor_ext',
 'tipoResolucion_ext',
 'termino_ext',
 'voto_salvado',
 'magistrados_ext',
 'Fechahora_ents',
 'Redactor_ents',
 'CitaSentencias_ents',
 'Reglamento_ents',
 'termino_lst',
 'subtema',
 'Cantón_ents',
 'Ley_ents',
 'EntidadPublica_ents',
 'subtema_prob',
 'Constitución_ents',
 'Fecha_ents',
 'anno',
 'Servicio_de_Agua',
 'Derecho_al_agua',
 'Contaminación_de_aguas',
 'parrafo',
 'No_corresponde',
 'legalidad']

In [ ]:
TAGS = ['ADJ','ADP','ADV','AUX','CONJ','CCONJ','DET','INTJ','PART','PRON','PUNCT','SCONJ']
INCLUIR = ['ADJ','ADV','NOUN','PROPN','VERB']
pattern1 = [{"POS": {"IN": ['NOUN', 'PROPN']}}, {"POS": {"IN": TAGS}, "OP": "*"}, {"POS": "VERB", "OP": "+"}]
pattern2 = [{"POS": "VERB"}, {"POS": {"IN": TAGS}, "OP": "*"}, {"POS": {"IN": ['NOUN', 'PROPN']}, "OP": "+"}]

from load_spacy import get_Spacy
nlp = get_Spacy('es_core_news_lg')
matcher = Matcher(nlp.vocab)
matcher.add("pattern", patterns)

matches = matcher(entr)       

for match_id, start, end in matches: